### Analyze residential delinquencies in Pittsburgh

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
CONDITIONDESC_RANKINGS = ['EXCELLENT', 'GOOD', 'AVERAGE', 'FAIR', 'POOR', 'UNSOUND', 'VERY POOR']
DB = create_engine('postgresql://jon@localhost:5432/propertydb')

# Methodology

### Run SQL query

In [3]:
properties = pd.read_sql_query("""SELECT usedesc, conditiondesc, COUNT(DISTINCT assessments.parcelid)
FROM assessments

JOIN parcelliens
ON parcelliens.parcelid = assessments.parcelid

JOIN parceldelinquenciespgh
ON parceldelinquenciespgh.parcelid = assessments.parcelid

JOIN parcelviolationspgh
ON parcelviolationspgh.parcelid = assessments.parcelid

WHERE assessments.municode::int < 200
AND assessments.class = 'R'
AND assessments.usedesc NOT IN ('VACANT LAND', 'RES AUX BUILDING (NO HOUSE)')
AND assessments.changenoticeaddress1 NOT LIKE '%%414%%GRANT%%ST%%'

AND parceldelinquenciespgh.prioryears::int >= 5

AND parcelviolationspgh.desc LIKE '%%VACANT%%'
AND parcelviolationspgh.outcome IN ('CONTINUE TO COURT', 'FOLLOW-UP INVESTIGATION SCHEDULED', 'ISSUE CRIMINAL COMPLAINT', 'REFER TO CLEANUP', 'SEND TO CLEAN & LIEN', 'VIOLATION FOUND')

GROUP BY usedesc, conditiondesc
ORDER BY usedesc, conditiondesc, count DESC;""", DB)

### Create pivot table by property use, condition

In [4]:
propertiespiv = pd.pivot_table(properties, index='usedesc', columns='conditiondesc', values='count', aggfunc=sum)

In [5]:
for ranking in CONDITIONDESC_RANKINGS:
    if ranking not in propertiespiv:
        propertiespiv[ranking] = 0
    propertiespiv[ranking] = propertiespiv[ranking].fillna(0)

propertiespiv = propertiespiv[CONDITIONDESC_RANKINGS]

In [6]:
propertiespiv

conditiondesc,EXCELLENT,GOOD,AVERAGE,FAIR,POOR,UNSOUND,VERY POOR
usedesc,,,,,,,
FOUR FAMILY,0,0,0.0,1.0,0.0,0.0,0.0
MOBILE HOME,0,0,0.0,0.0,1.0,0.0,0.0
ROWHOUSE,0,0,9.0,24.0,7.0,5.0,1.0
SINGLE FAMILY,0,0,147.0,182.0,48.0,20.0,11.0
THREE FAMILY,0,0,1.0,5.0,0.0,0.0,0.0
TOWNHOUSE,0,0,3.0,0.0,0.0,0.0,0.0
TWO FAMILY,0,0,10.0,11.0,3.0,2.0,1.0


In [7]:
propertiespiv.sum().sum()

492.0

### Output properties to CSV file

In [8]:
propertiespiv.to_csv('output/residential-delinquencies.csv')